# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

В самом начале импортируем все необходимые библиотеки

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score #

import numpy as np

from scipy import stats as st

В нашем распоряжении три отдельных файла с данными по трем разным регионам. Каждую таблицу посмотрим отдельно и если будет необходимо, то проведем предобработку.

### Загрузка и подготовка данных первый регион

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv') # открытие первого файла

In [3]:
display(geo_data_0.head()) # просмотр первых пяти строк.

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


На первый взгляд, ошибок в данных не видно. В файле есть показатели f0, f1, f2, которые характеризуют свойства залежей, но поскольку нам не уточняют, что это конкретно, будем считать, что данные корректны и они могут действительно быть как положительные, так и отрицательные. Данную информацию, лучше уточнить у специалистов, но как я уже сказала в данном случае считам данные верными. Посмотрим информацию о файле методом info()

In [4]:
geo_data_0.info() # просмотр информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В файле 4 столбца. В одном столбце тип данных - object, а в трех остальных- float64. Необходимости менять тип данных в столбах нет. Во всех столбцах одинаковое количество данных, следовательно пропусков нет.

В соответствии с документацией в файле есть следующие данные:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).

Проверим, нет ли полностью дублирующихся строк:

In [5]:
geo_data_0.duplicated().sum() # подсчет количества дублирующихся строк

0

Полностью дублирующихся строк нет. Посмотрим, нет ли дублирующихся "id":

In [6]:
geo_data_0['id'].value_counts().head(15) # подсчет количества дублирующихся уникальных значений на примере первых 15 позиций

Tdehs    2
bxg6G    2
AGS9W    2
A5aEY    2
bsk9y    2
HZww2    2
fiKDv    2
74z30    2
TtcGQ    2
QcMuo    2
1KiCT    1
D6Nj9    1
3fsYF    1
2KS4I    1
QJ3jY    1
Name: id, dtype: int64

In [7]:
display(geo_data_0.loc[geo_data_0['id']=='QcMuo']) # с помощью логической индексации вывод строк по заданному условию 

,id,f0,f1,f2,product
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675


Поскольку на количество запасов нефти влияют, в данном случае только показатели f0-f3, обращать внимание на дублирующийся ID мы не будем. Скорее всего анализ скважины проводили несколько раз, поэтому ее id продублировался.

Посмотрим насколько сильно коррелируют между собой данные в таблице. Если корреляция между какими-то столбцами будет очень сильной, то это может нам помешать построить модель, которая будет адекватно предсказывать результаты.

In [8]:
geo_data_0.corr() # расчет попарного коэффициента корреляции

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Корреляция достаточно слабая, так что можно спокойно использовать все столбцы для построения модели. Дополнительно посмотрим статистические данные по столбцам с признаками.

In [9]:
geo_data_0[['f0', 'f1', 'f2', 'product']].describe() # просмотр статистических данных по столбцам с признаками

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


По всем столбцам медиана и среднее приблизительно равны, и можно говорить о нормальном распределении. Максимальные и минимальные значения по всем столбцам в одном диапазоне, поэтому масштабировать признаки не требуется.

Далее посмотрим данные из второго набора данных.

### Загрузка и подготовка данных второй регион

In [10]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv') # открытие второго файла

In [11]:
display(geo_data_1.head()) # просмотр первых пяти строк.

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


На первый взгляд, ошибок в данных не видно. Посмотрим информацию о файле методом info()

In [12]:
geo_data_1.info() # просмотр информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В файле 4 столбца. В одном столбце тип данных - object, а в трех остальных- float64. Необходимости менять тип данных в столбах нет. Во всех столбцах одинаковое количество данных, следовательно пропусков нет.

В соответствии с документацией в файле есть следующие данные:

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).

In [13]:
geo_data_1.duplicated().sum() # подсчет количества дублирующихся строк

0

Полностью дублирующихся строк нет. Проверять есть ли дублирующиеся данные в столбце с "id" мы не будем, потому что как мы сказали ранее видимо анализ в некоторых скважинах проводился несколько раз, но данные были получены разные и их признали релевантными. Последнее, что мы посмотрим - это корреляцию.

In [14]:
geo_data_1.corr() # расчет попарного коэффициента корреляции

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Корреляция между признаками f0-f2 достаточно низкая. Данные пригодны для построения модели.

In [15]:
geo_data_1[['f0', 'f1', 'f2', 'product']].describe() # просмотр статистических данных по столбцам с признаками

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


Здеть ситуация аналогичная данным по первому региону: распределение признаков близко к нормальному. Данные находятся в одном числовом диапазоне, поэтому масштабирование не требуется.

Посмотрим последний комплект данных.

### Загрузка и подготовка данных третий регион

In [16]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv') # открытие третьего файла

In [17]:
display(geo_data_2.head()) # просмотр первых пяти строк.

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


На первый взгляд, ошибок в данных не видно. Посмотрим информацию о файле методом info()

In [18]:
geo_data_2.info() # просмотр информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


В файле 4 столбца. В одном столбце тип данных - object, а в трех остальных- float64. Необходимости менять тип данных в столбцах нет. Во всех столбцах одинаковое количество данных, следовательно пропусков нет. Состав данных такой же как и в первых двух таблицах.

In [19]:
geo_data_2.duplicated().sum() # подсчет количества дублирующихся строк

0

Полностью дублирующихся строк нет. Посмотрим корреляцию между признаками.

In [20]:
geo_data_2.corr() # расчет попарного коэффициента корреляции

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Корреляция между признаками f0-f2 достаточно низкая. Данные пригодны для построения модели.

In [21]:
geo_data_2[['f0', 'f1', 'f2', 'product']].describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Данные находятся в одном числовом диаразоне, поэтому масштабироавние не требуется. А вот распределение нормальное только у признака F2. Ву всех остальных признаков наблюдается скошенность (у F0 скошенность влево, а у F1 - вправо).

**Вывод**

Мы посмотрели все три таблицы. Состав данных в них одинаковый. Явных ошибок в данных мы не обнаружили. Подностью дублирующихся строк нет. Пропусков нет. Коэффициент корреляции между признаками f0-f2 низкий, а значит можно строить можель не боясь искажения результата. Признаки находятся в одном диапазоне, а значит масштабирование проводить не нужно. 

Данные готовы для построения моделей.

## Обучение и проверка модели

В этом разделе мы построим модель, которая будет предсказывать объем запасов нефти в новых скважинах. 

У нас данные из трех разных регионов, а как известно, нет двух одинаковых месторождений. На формирование залежей влияют разные условия: характер природного резервуара, толщина и прочность/пористость пород, кавернозность горных пород, давление внутри залежи и многое другое. Также соотношение газа, нефти и воды в пустотных пространствах в разных местах разное. Поэтому нельзя строить одну модель для всех регионов. Мы обучим для каждого региона свою модель и проверим ее на валидационной выборке.

### Построение модели для первого региона

Разделим выборку на признаки (features) и целевой признак (target). В качестве целевого признака у нас будет объем запасов в скважине.

In [22]:
target_0 = geo_data_0['product'] # целевой признак
features_0 = geo_data_0.drop('product', axis=1) # признаки


Для начала нам необходимо разделить данные на две части: на обучающую и валидационную выборки. Обучающая выборка будет самой большой и вместит 75 % данных. С помощью нее мы будем обучать модель. Вторая выборка валидационная (25% от объема данных), нужна нам для проверки качества работы алгоритма во время обучения модели.

In [23]:
features_train_0, features_valid_0, target_train_0, \
 target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345) # разделение данных на обучаюший и валидационные наборы данных

Проверим, что разделили верно:

In [24]:
print(features_train_0.shape)
print(target_train_0.shape)

(75000, 4)
(75000,)


In [25]:
print(features_valid_0.shape)
print(target_valid_0.shape)

(25000, 4)
(25000,)


Поскольку целевой признак у нас количественный, то нам придется решать задачу регрессии. Мы будем обучать модель линейной регрессии, как наиболее предсказуемую.

In [26]:
model_0 = LinearRegression() # инициализация модели линейной регрессии
model_0.fit(features_train_0[['f0', 'f1', 'f2']], target_train_0) # обучение модели
predicted_valid_0 = model_0.predict(features_valid_0[['f0', 'f1', 'f2']]) # получение предсказания результата на валидационной выборке

In [27]:
predicted_valid_0 = pd.Series(predicted_valid_0, index = target_valid_0.index) 
# преобразование предсказаний в объект Series и присвоение значениям индексов, как у целевого признака

В качестве метрики качества для своей модели мы будем использовать RMSE - корень из среднеквадратичной ошибки, который показывает как сильно правильный ответ отличается от предсказания.

In [28]:
rmse =  (mean_squared_error(target_valid_0, predicted_valid_0)) ** 0.5 # расчет корня из среднеквадратичной ошибки

print("Linear Regression")
print("RMSE:", rmse)


Linear Regression
RMSE: 37.5794217150813


Чтобы оценить адекватность модели, нужно сравнить её со случайной. Отвечать на все объекты одним и тем же числом — простой способ регрессионного предсказания. Чтобы оно не сильно расходилось с правдой, примем за такое число среднее значение.

In [29]:
predicted_valid = pd.Series(target_train_0.mean(), index=target_valid_0.index)
rmse = (mean_squared_error(target_valid_0, predicted_valid)) ** 0.5

print("Mean")
print("RMSE =", rmse)

Mean
RMSE = 44.289591053907365


Чем меньше RSME, тем лучше модель предсказывает. В нашем случае, этот показатель ниже, чем RSME случайной модели - это говорит о том, что модель неплохо справляется.

Посчитаем среднее зачение объема запасов нефти по предсказанным данным и реальным.

In [30]:
print("Среднее значение объема запасов (реальные данные):", target_valid_0.mean())
print("Среднее значение объема запасов (предсказанные данные):", predicted_valid_0.mean())

Среднее значение объема запасов (реальные данные): 92.07859674082927
Среднее значение объема запасов (предсказанные данные): 92.59256778438035


Среднии значения достаточно близки, но модель предсказывает немного более оптимистичные данные, чем есть на самом деле.

Для дополнительной проверки качества модели посчитаем такой показатель как коэффициент детерминации, или метрика R2.

In [31]:
print("R2 =", r2_score(target_valid_0, predicted_valid_0)) # расчет коэфициента детирминации

R2 = 0.27994321524487786


Чем ближе этот показатель к единице, тем лучше. В нашем случает модель работает лучше, чем среднее, но как ранее мы видили, что до идеала ей далеко. 

### Построение модели для второго региона

Аналогично, как и для первого региона разделим выборку на признаки (features) и целевой признак (target). В качестве целевого признака у нас будет объем запасов в скважине.

In [32]:
target_1 = geo_data_1['product'] # целевой признак
features_1 = geo_data_1.drop('product', axis=1) # признаки


Поделим данные на две части: на обучающую и валидационную выборки. Обучающая выборка будет самой большой и вместит 75 % данных. С помощью нее мы будем обучать модель. Вторая выборка валидационная (25% от объема данных), нужна нам для проверки качества работы алгоритма во время обучения модели.

In [33]:
features_train_1, features_valid_1, target_train_1, \
 target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345) # разделение данных на обучаюший и валидационные наборы данных

Проверим, что разделили верно:

In [34]:
print(features_train_1.shape)
print(target_train_1.shape)

(75000, 4)
(75000,)


In [35]:
print(features_valid_1.shape)
print(target_valid_1.shape)

(25000, 4)
(25000,)


Поскольку целевой признак у нас количественный то нам придется решать задачу регрессии. Мы будем обучать модель линейной регрессии, как наиболее предсказуемую.

In [36]:
model_1 = LinearRegression() # инициализация модели линейной регрессии
model_1.fit(features_train_1[['f0', 'f1', 'f2']], target_train_1) # обучение модели
predicted_valid_1 = model_1.predict(features_valid_1[['f0', 'f1', 'f2']]) # получение предсказания результата на валидационной выборке

In [37]:
predicted_valid_1 = pd.Series(predicted_valid_1, index = target_valid_1.index)
# преобразование предсказаний в объект Series и присвоение значениям индексов, как у целевого признака

В качестве метрики качества для своей модели мы будем использовать RMSE - корень из среднеквадратичной ошибки, который показывает как сильно правильный ответ отличается от предсказания.

In [38]:
rmse =  (mean_squared_error(target_valid_1, predicted_valid_1)) ** 0.5 # расчет корня из среднеквадратичной ошибки

print("Linear Regression")
print("RMSE:", rmse)


Linear Regression
RMSE: 0.893099286775617


Чтобы оценить адекватность модели, нужно сравнить её со случайной. Отвечать на все объекты одним и тем же числом — простой способ регрессионного предсказания. Чтобы оно не сильно расходилось с правдой, примем за такое число среднее значение.

In [39]:
predicted_valid = pd.Series(target_train_1.mean(), index=target_valid_1.index)
rmse = (mean_squared_error(target_valid_1, predicted_valid)) ** 0.5

print("Mean")
print("RMSE =", rmse)

Mean
RMSE = 46.02144533725462


Чем меньше RSME тем лучше модель предсказывает. В нашем случае, этот показатель значительно ниже, RSME случайной модели - это говорит о том, что модель очень хорошо работает.

Посчитаем среднее зачение объема запасов нефти по предсказанным данным и реальным.

In [40]:
print("Среднее значение объема запасов (реальные данные):", target_valid_1.mean())
print("Среднее значение объема запасов (предсказанные данные):", predicted_valid_1.mean())

Среднее значение объема запасов (реальные данные): 68.72313602435997
Среднее значение объема запасов (предсказанные данные): 68.728546895446


Средние значения практически равны.

Для дополнительной проверки качества модели посчитаем такой показатель как коэффициент детерминации, или метрика R2.

In [41]:
print("R2 =", r2_score(target_valid_1, predicted_valid_1))

R2 = 0.9996233978805127


Чем ближе этот показатель к единице, тем лучше. В нашем случает модель работает практически идеально.

### Построение модели для третьего региона

Снова разделим выборку на признаки (features) и целевой признак (target). В качестве целевого признака у нас будет объем запасов в скважине.

In [42]:
target_2 = geo_data_2['product'] # целевой признак
features_2 = geo_data_2.drop('product', axis=1) # признаки


Для начала нам необходимо разделить данные на две части: на обучающую и валидационную выборки. Обучающая выборка будет самой большой и вместит 75 % данных. С помощью нее мы будем обучать модель. Вторая выборка валидационная (25% от объема данных), нужна нам для проверки качества работы алгоритма во время обучения модели.

In [43]:
features_train_2, features_valid_2, target_train_2, \
 target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345) # разделение данных на обучаюший и валидационные наборы данных

Проверим, что разделили верно:

In [44]:
print(features_train_2.shape)
print(target_train_2.shape)

(75000, 4)
(75000,)


In [45]:
print(features_valid_2.shape)
print(target_valid_2.shape)

(25000, 4)
(25000,)


Вновь целевой признак у нас количественный и нам придется решать задачу регрессии. И снова мы будем обучать модель линейной регрессии, как наиболее предсказуемую.

In [46]:
model_2 = LinearRegression() # инициализация модели линейной регрессии
model_2.fit(features_train_2[['f0', 'f1', 'f2']], target_train_2) # обучение модели
predicted_valid_2 = model_2.predict(features_valid_2[['f0', 'f1', 'f2']]) # получение предсказания результата на валидационной выборке

In [47]:
predicted_valid_2 = pd.Series(predicted_valid_2, index = target_valid_2.index)
# преобразование предсказаний в объект Series и присвоение значениям индексов, как у целевого признака

В качестве метрики качества для своей модели мы будем использовать RMSE - корень из среднеквадратичной ошибки, который показывает как сильно правильный ответ отличается от предсказания.

In [48]:
rmse =  (mean_squared_error(target_valid_2, predicted_valid_2)) ** 0.5 # расчет корня из среднеквадратичной ошибки

print("Linear Regression")
print("RMSE:", rmse)


Linear Regression
RMSE: 40.02970873393434


Как и в предыдущих двух случаях, оценим адекватность модели, сравнив её со случайной. 

In [49]:
predicted_valid = pd.Series(target_train_2.mean(), index=target_valid_2.index)
rmse = (mean_squared_error(target_valid_2, predicted_valid)) ** 0.5

print("Mean")
print("RMSE =", rmse)

Mean
RMSE = 44.90234968510566


Чем меньше RSME тем лучше модель предсказывает. В нашем случае, этот показатель немного ниже, чем RSME случайной модели - это говорит о том, что и эта модель неплохо справляется.

Посчитаем среднее зачение объема запасов нефти по предсказанным данным и реальным.

In [50]:
print("Среднее значение объема запасов (реальные данные):", target_valid_2.mean())
print("Среднее значение объема запасов (предсказанные данные):", predicted_valid_2.mean())

Среднее значение объема запасов (реальные данные): 94.88423280885438
Среднее значение объема запасов (предсказанные данные): 94.96504596800489


Среднии значения достаточно близки, но модель предсказывает немного более оптимистичные данные, чем есть на самом деле.

Для дополнительной проверки качества модели, посчитаем такой показатель, как коэффициент детерминации (метрика R2).

In [51]:
print("R2 =", r2_score(target_valid_2, predicted_valid_2))

R2 = 0.20524758386040443


Чем ближе этот показатель к единице, тем лучше. В нашем случает модель работает лучше, чем среднее, но как ранее мы видим, что до идеала ей далеко. 

**Вывод**

В этом разделе мы построили для каждого региона свою модель линейной регрессии. Во всех трех случаях среднее значение предсказанных объемов близко к среднему реальных объемов запасов в скважине. Вторая модель обучилась лучше, чем первая и третья. Для всех трех моделей мы посчитали показатель RSME, который показывает на сколько сильно правильный ответ отличается от предсказания. Все расчитанные RSME ниже RSME случайных моделей. Дополнительно мы посчитали коэффициент детерминации, который подтвердил наше предположение о качестве моделей: лучше всего работает модель для второго региона (RSME = 0.89, R2 = 0.99), на втором месте модель для первого региона (RSME = 37.58, R2 = 0.28), на третьем месте модель для третьего региона (RSME = 40.03,  R2 = 0.21

## Подготовка к расчёту прибыли

Для начала сохраним в переменные значения выручки и затрат, чтобы если в последствии эти показатели придется менять, то поменять их нужно будет только в одном месте.

In [52]:
TOTAL_BUDGET = 10000000000 # общий бюджет затрат на разработку 200 скважин
TOTAL_WELLS = 200 # количество разрабатываемых скважин
REVENUE_PER_UNIT = 450000 # выручка за тысячу баррелей нефти



Теперь определимся, что такое точка безубыточности. Точкой безубыточности называют такой объем продаж/реализации, который полностью покрывает затраты, но прибыль при этом равна нулю. Исходя из этого определения рассчитаем точку безубыточности или как еще называют порог рентабельности в среднем для одной скважины.

In [53]:
break_even_point = TOTAL_BUDGET / REVENUE_PER_UNIT / TOTAL_WELLS # расчет точки безубыточности для одной скважины
print(break_even_point)

111.11111111111111


Сравним полученное значение со средней добычей по регионам.

In [54]:
print(break_even_point <= target_0.mean())
print(break_even_point <= target_1.mean())
print(break_even_point <= target_2.mean())

False
False
False


Во всех трех регионах среднее количество добычи меньше реального среднего добываемого количества. Теперь посмотрим насколько реальное среднее отличается от среднего безубыточного объема.

In [55]:
print("Средний безубыточный объем запасов больше реального среднего запаса в регионе 1 на:", \
      break_even_point - target_0.mean())
print("Средний безубыточный объем запасов больше реального среднего запаса в регионе 2 на:", \
      break_even_point - target_1.mean())
print("Средний безубыточный объем запасов больше реального среднего запаса в регионе 3 на:", \
      break_even_point - target_2.mean())

Средний безубыточный объем запасов больше реального среднего запаса в регионе 1 на: 18.6111111111111
Средний безубыточный объем запасов больше реального среднего запаса в регионе 2 на: 42.2861111111111
Средний безубыточный объем запасов больше реального среднего запаса в регионе 3 на: 16.11111111111107


Ближе всего к точке безубыточности третий регион. 

**Вывод**

В этом разделе мы посчитали средний объем добычи нефти, который позволит нам достигнуть точки безубыточности, т.е такой объем при котором выручка будет полностью покрывать затраты, но прибыль будет равна нулю. Далее мы сравнили этот показатель со средней добычей по регионам и оказалось, что расчитанные реальные средние ниже точки безубыточности. Ближе всего к безубыточному объему третий регион, где реальная средняя меньше точки безубыточности на 16,11 тыс.баррелей, а дальше всего второй (разница составляет 42,28 тыс.баррелей). 

## Расчёт прибыли и рисков 

В этом разделе мы посчитаем среднюю прибыль по регионам применив технику Bootstrap с 1000 выборками, а также посчитаем риск убытков.

Для начала создадим функцию, которая будет рассчитывать прибыль, основываясь на предсказаниях сделанных моделью.

In [56]:
def profit (target_valid, predicted_valid, count):
    prob_sorted = predicted_valid.sort_values(ascending = False) # сохраняем в переменную отсортированный по убыванию список значений предсказаний
    selected = target_valid[prob_sorted.index][:count] # отбираем значения из целевого признака по индексу предсказаний
   
        
    return REVENUE_PER_UNIT* selected.sum() - TOTAL_BUDGET # формула расчета прибыль: цена * кол-во - затраты



Далее применим Bootstrap с 1000 выборками для моделей. Эта техника сводится к тому, что из большой выборки случайно отбираются элементы в подвыборки. Мы будем проводить отбор с возвращением, т.е. один и тот же элемнт может быть в разных выборках. Далее в каждой подвыборке производятся вычисления (мы будем считать прибыль для 200 лучших элементов из 500). Далее мы посчитаем среднюю прибыль по каждому региону основываясь на данных полученных с помощью Bootstrap.

Также мы найдем крайние значения доверительного интервала равного 95% и посчитаем риск убытков. Риск убытков мы расчитаем следующим образом: количество значений убытка разделим на общее количество рассчитанных значений прибыли/убытка с помощью техники Bootstrap

In [57]:
state = np.random.RandomState(12345) #  указываем random_state для воспроизводимости

Рассчитаем прибыль и риски для первого региона.

In [58]:
values_0 = []
for i in range(1000):
    target_subsample = target_valid_0.sample(n=500, frac=None, replace=True, random_state=state) # создание подвыборок
       
    probs_subsample = predicted_valid_0[target_subsample.index] # подбор значений предсканий по индекцу целевого признака попавшего в подвыборку
    values_0.append(profit(target_subsample, probs_subsample, 200)) # расчет значения прибыли для 200 скважин
    
values_0 = pd.Series(values_0)
mean = values_0.mean() # расчет средней прибыли по региону

lower_0 = values_0.quantile(0.025) 
upper_0 = values_0.quantile(0.975) # расчет доверительного интервала

risk_of_loss_0 = (values_0 < 0).mean() #расчет рисков убытков

print("Средняя прибыль в первом регионе:", mean)
print("Доверительный интервал:", lower_0, upper_0)
print("Риск убытков в первом регионе составляет:", risk_of_loss_0)


Средняя прибыль в первом регионе: 425938526.91059244
Доверительный интервал: -102090094.83793654 947976353.3583689
Риск убытков в первом регионе составляет: 0.06


Рассчитаем прибыль и риски для второго региона.

In [59]:
values_1 = []
for i in range(1000):
    target_subsample = target_valid_1.sample(n=500, frac=None, replace=True, random_state=state) # создание подвыборок
       
    probs_subsample = predicted_valid_1[target_subsample.index] # подбор значений предсканий по индекцу целевого признака попавшего в подвыборку
    values_1.append(profit(target_subsample, probs_subsample, 200)) # расчет значения прибыли для 200 скважин
    
values_1 = pd.Series(values_1)
mean = values_1.mean() # расчет средней прибыли по региону

lower_1 = values_1.quantile(0.025) 
upper_1 = values_1.quantile(0.975) # расчет доверительного интервала

risk_of_loss_1 = (values_1 < 0).mean() #расчет рисков убытков


print("Средняя прибыль во втором регионе:", mean)
print("Доверительный интервал:", lower_1, upper_1)
print("Риск убытков во втором регионе составляет:", risk_of_loss_1)

Средняя прибыль во втором регионе: 518259493.6973249
Доверительный интервал: 128123231.43308444 953612982.0669085
Риск убытков во втором регионе составляет: 0.003


Рассчитаем прибыль и риски для первого региона.

In [60]:
values_2 = []
for i in range(1000):
    target_subsample = target_valid_2.sample(n=500, frac=None, replace=True, random_state=state) # создание подвыборок
       
    probs_subsample = predicted_valid_2[target_subsample.index] # подбор значений предсканий по индекцу целевого признака попавшего в подвыборку
    values_2.append(profit(target_subsample, probs_subsample, 200)) # расчет значения прибыли для 200 скважин
    
values_2 = pd.Series(values_2)
mean = values_2.mean() # расчет средней прибыли по региону

lower_2 = values_2.quantile(0.025) 
upper_2 = values_2.quantile(0.975) # расчет доверительного интервала

risk_of_loss_2 = (values_2 < 0).mean() #расчет рисков убытков


print("Средняя прибыль в третьем регионе:", mean)
print("Доверительный интервал:", lower_2, upper_2)
print("Риск убытков в третьем регионе составляет:", risk_of_loss_2)

Средняя прибыль в третьем регионе: 420194005.3440501
Доверительный интервал: -115852609.16001143 989629939.8445739
Риск убытков в третьем регионе составляет: 0.062


Итак результаты у нас такие:
- 1 регион: средняя прибыль 425,9 млн.руб, доверительный интервал в диапазоне от -102.1 млн.руб. до 948.0 млн.руб., риск убытков 6%
- 2 регион: средняя прибыль 518,3 млн.руб, доверительный интервал в диапазоне от 128.0 млн.руб. до 953.6 млн.руб., риск убытков 0,3%
- 3 регион: средняя прибыль 420,2 млн.руб, доверительный интервал в диапазоне от -115.9 млн.руб. до 989.6 млн.руб., риск убытков 6,2%


## Общий вывод

В данной работе мы проводили исследование для определения региона, где добыча нефти принесет наибольшую прибыль.

В первом разделе работы мы загрузили и изучили данные по трем регионам. В нашем распоряжении данные о 100 тыс.скважин в каждом регионе. Пропусков в данных нет, дубликатов строк тоже. Все данные по признакам находятся в одном числовом диапазоне, поэтому масштабировать признаки не нужно. Данные пригодны для построения моделей.

Во втором разделе мы обучили три модели линейной регрессии, по одной на каждый регион. Делать одну модель на всех не целесообразно, поскольку характеристики нефтяных залежей в разных регионах разные. Линейную регрессию выбрали как наиболее предсказуемый вид модели.

Для всех трех моделей мы посчитали показатель RSME и дополнительно мы посчитали коэффициент детерминации, который подтвердил наше предположение о качестве моделей: лучше всего работает модель для второго региона (RSME = 0.89, R2 = 0.99), на втором месте модель для первого региона (RSME = 37.58, R2 = 0.28), на третьем месте модель для третьего региона (RSME = 40.03, R2 = 0.21).

В следующем разделе мы посчитали безубыточный объем добычи нефти с одной скважины и сравнили его со средним запасом в каждом регионе. Ближе всего к безубыточному объему третий регион, где реальная средняя меньше точки безубыточности всего на 16,11 тыс.баррелей, а дальше всего второй (разница составляет 42,28 тыс.баррелей). 

Поскольку для разведки региона исследуют 500 точек и только 200 лучших выбирают для разработки, мы применили технику Bootstrap с 1000 выборками и посчитали среднюю прибыль по этим выборкам в четвертом разделе проекта. Результаты получились следующие:
- 1 регион: средняя прибыль 425,9 млн.руб, доверительный интервал в диапазоне от -102.1 млн.руб. до 948.0 млн.руб., риск убытков 6%
- 2 регион: средняя прибыль 518,3 млн.руб, доверительный интервал в диапазоне от 128.0 млн.руб. до 953.6 млн.руб., риск убытков 0,3%
- 3 регион: средняя прибыль 420,2 млн.руб, доверительный интервал в диапазоне от -115.9 млн.руб. до 989.6 млн.руб., риск убытков 6,2%

По результатам проведенного исследования рекомендую для разработки второй регион, потому что в нем риск убытков небольшой, а средняя прибыль самая большая из всех трех регионов.